# Some context on what's being developed
### Remarks:
 - A minimal setup should be implemented first
 - Should point to a more extensive mini-language approach (for the future)

### Linting workflow through magics:
- First, load up the magics:
```ipython
%reload_ext datalintermagics 
```
- **Data setup** (line magic) i.e. , specify dataset plus arguments for passing it to the linter it i.e. `header`, `delimiter`:
```ipython
%add_linter_data --tracked-variable my_data_var --data-delim=',' --data-header=True
```
- **Lint command** (cell magic) i.e. send code and the tracked variable to the linter, get reply back and show it:
```ipython
%%lint --ip 0.0.0.0 --port 10000 --show-stats True --show-na False --show-passing False
```
   
### Shortlist of TODOs concerning magics:
 - track more than one variable (defining meaning there)
 - add support for linting lines i.e.

In [15]:
%reload_ext datalintermagics
b = 2
%add_linter_data --tracked-variable b --data-header True --data-delim ','

Warning (Linter): Unsupported type <class 'int'> for 'b'. Supported: 'numpy.ndarray', 'pandas.DataFrame'.


In [16]:
import matplotlib.pyplot as plt

# Though the following import is not directly being used, it is required
# for 3D projection to work with matplotlib < 3.2
import mpl_toolkits.mplot3d  # noqa: F401
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.cluster import KMeans
np.random.seed(5)

In [17]:
from sklearn import datasets
iris = datasets.load_iris()
iris_df = pd.DataFrame(
            np.hstack(
                (iris.data, 
                 np.reshape(iris.target,(len(iris.target),1))
                )
            ))
iris_df.columns = [*iris.feature_names, 'target']

# Add data variable to linter with line magic
%add_linter_data --tracked-variable iris_df --data-header 'True' --data-delim ','

>>DEBUG: Added 'iris_df' as linter data.


### Lint with %%lint cell magic

In [18]:
%%lint --ip 0.0.0.0 --port 10000 --show-stats True --show-na False --show-passing False
# This simply lints the data, no code is available for the cell

Response: 200, OK
Linter output
-------------
• info         	(enum_detector)     	column: target       just a few distinct values in 'column: target', it could be an enum
1 issue found from 18 linters applied (6 OK, 12 N/A) .



### Lint (data and code simultaneously)

In [19]:
%%lint --ip 0.0.0.0 --port 10000 --show-stats True --show-na False --show-passing False
# This will send code as well to the linter
X = iris.data
y = iris.target

%%add_linter_data
estimators = [
    ("k_means_iris_8", KMeans(n_clusters=8)),
    ("k_means_iris_3", KMeans(n_clusters=3)),
    ("k_means_iris_bad_init", KMeans(n_clusters=3, n_init=1, init="random")),
]

fig = plt.figure(figsize=(10, 8))
titles = ["8 clusters", "3 clusters", "3 clusters, bad initialization"]
for idx, ((name, est), title) in enumerate(zip(estimators, titles)):
    ax = fig.add_subplot(2, 2, idx + 1, projection="3d", elev=48, azim=134)
    est.fit(X)
    labels = est.labels_

    ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=labels.astype(float), edgecolor="k")

    ax.xaxis.set_ticklabels([])
    ax.yaxis.set_ticklabels([])
    ax.zaxis.set_ticklabels([])00
    ax.set_xlabel("Petal width")
    ax.set_ylabel("Sepal length")
    ax.set_zlabel("Petal length")
    ax.set_title(title)

# Plot the ground truth
ax = fig.add_subplot(2, 2, 4, projection="3d", elev=48, azim=134)

for name, label in [("Setosa", 0), ("Versicolour", 1), ("Virginica", 2)]:
    ax.text3D(
        X[y == label, 3].mean(),
        X[y == label, 0].mean(),
        X[y == label, 2].mean() + 2,
        name,
        horizontalalignment="center",
        bbox=dict(alpha=0.2, edgecolor="w", facecolor="w"),
    )

ax.scatter(X[:, 3], X[:, 0], X[:, 2], c=y, edgecolor="k")

ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])
ax.set_xlabel("Petal width")
ax.set_ylabel("Sepal length")
ax.set_zlabel("Petal length")
ax.set_title("Ground Truth")

plt.subplots_adjust(wspace=0.25, hspace=0.25)
plt.show()

Response: 200, OK
Linter output
-------------
• info         	(enum_detector)     	column: target       just a few distinct values in 'column: target', it could be an enum
1 issue found from 18 linters applied (6 OK, 12 N/A) .



In [20]:
# Add data variable to linter with line magic
X = iris.data
%add_linter_data --tracked-variable X --data-header 'False' --data-delim ','

>>DEBUG: Added 'X' as linter data.


In [21]:
%%lint --ip 0.0.0.0 --port 10000 --show-stats True --show-na False --show-passing False
##

Response: 200, OK
Linter output
-------------
0 issues found from 18 linters applied (6 OK, 12 N/A) .

